# Capítulo 3. Algoritmos genéticos.
> Autor: Natalia Cely Callejas, Ronald Chavez.\
> Universidad Nacional de Colombia  
> Mayo 1, 2025

## Introducción 
Los algoritmos genéticos son herramientas que emulan los ciclos de vida y los comportamientos genétidos de los seres humanos, desarrollados entre 1960 y 1970, describen el flujo de la evolución humana que siguen:
1. Selección. De los organismos (soluciones) más aptos.
2. Mutación. Posibilidad de cambios para mantener la diversidad.
3. Cruce. Las características de los seleccionados se transmiten a la siguiente generación.

Bajo el marco de una serie de reglas que lo condicionan concordantes con la aplicación, así, los Algoritmos genéticos son utilizados para problemas de combinación y optimización, sobretodo en situaciones de altos órdenes o complejidad, cuando no existe una forma directa para abarcarlos. 

## Verdadera Democracia



## Optimización de despacho de energía en relación con los costos de transporte
La situación estudiada es la siguiente:
> Una empresa proveedora de energía eléctrica dispone de 4 plantas de generación para satisfacer la demanda diaria de energía eléctrica en Cali, Bogotá, Medellín y Barranquilla. Cada una puede generar 3, 6, 5 y 4 GW al día respectivamente. Las necesidades de Cali, Bogotá, Medellín y Barranquilla son de 4, 3, 5 y 3 GW al día respectivamente. Lo costos por el transporte de energía por cada GW entre plantas y ciudades se dan en la siguente tabla:


|        |Cali|Bogotá|Medellín|Barranquilla|
|--------|----|------|--------|------------|
|Planta A|1   |4     |3       |6           |
|Planta B|4   |1     |4       |5           |
|Planta C|3   |4     |1       |4           |
|Planta D|6   |5     |4       |1           |

> Y los costos de $KW-H$ por generador se dan en la siguiente tabla:

|Generador | KW-H |
|----------|------|
|Planta A  | 680|
|Planta B  | 720|
|Planta C  |660 |
|Planta D  | 750|